In [5]:
import cv2
import torch
import numpy as np
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.metrics.pairwise import cosine_similarity
from deepface import DeepFace

# Initialize models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').to(device).eval()

# Paths
KNOWN_FACES_DIR = r"C:\Users\afnan\Desktop\known_faces"
SIMILARITY_THRESHOLD = 0.6
MAX_FACES_PER_FRAME = 5

# Load known faces
known_faces = {}
for person in os.listdir(KNOWN_FACES_DIR):
    person_dir = os.path.join(KNOWN_FACES_DIR, person)
    if os.path.isdir(person_dir):
        embeddings = []
        for img_file in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            boxes, _ = mtcnn.detect(img_rgb)
            if boxes is not None and len(boxes) > 0:
                box = boxes[0]
                x1, y1, x2, y2 = map(int, box)
                face = img[y1:y2, x1:x2]
                if face.size == 0:
                    continue
                face_resized = cv2.resize(face, (160, 160))
                face_resized = face_resized.astype('float32') / 255.0
                face_tensor = torch.from_numpy(face_resized.transpose(2, 0, 1)).unsqueeze(0).to(device)
                with torch.no_grad():
                    emb = resnet(face_tensor).cpu().numpy()
                embeddings.append(emb.flatten())
        if embeddings:
            mean_emb = np.mean(embeddings, axis=0)
            known_faces[person] = {'embedding': mean_emb, 'count': len(embeddings)}
            print(f"Loaded {len(embeddings)} images for {person}")

print(f"\nLoaded {len(known_faces)} persons.")

# Video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = mtcnn.detect(frame_rgb)
    
    if boxes is not None:
        unknown_embeddings = []

        for box in boxes[:MAX_FACES_PER_FRAME]:
            x1, y1, x2, y2 = map(int, box)
            face = frame[y1:y2, x1:x2]
            
            if face.size == 0:
                continue
                
            # Face recognition processing
            face_resized = cv2.resize(face, (160, 160))
            face_resized = face_resized.astype('float32') / 255.0
            face_tensor = torch.from_numpy(face_resized.transpose(2, 0, 1)).unsqueeze(0).to(device)
            
            with torch.no_grad():
                emb = resnet(face_tensor).cpu().numpy().flatten()

            # Check for duplicate faces
            is_duplicate = False
            for known_emb in unknown_embeddings:
                if cosine_similarity([emb], [known_emb])[0][0] > 0.8:
                    is_duplicate = True
                    break

            if not is_duplicate:
                name = "Unknown"
                best_score = -1
                for person, data in known_faces.items():
                    score = cosine_similarity([emb], [data['embedding']])[0][0]
                    if score > best_score:
                        best_score = score
                        if score >= SIMILARITY_THRESHOLD:
                            name = person

                if name == "Unknown":
                    unknown_embeddings.append(emb)

                # Emotion detection using DeepFace
                emotion = "N/A"
                try:
                    face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                    if face_rgb.shape[0] >= 64 and face_rgb.shape[1] >= 64:
                        emotion_result = DeepFace.analyze(
                            face_rgb, 
                            actions=['emotion'],
                            detector_backend='opencv',
                            enforce_detection=False,
                            silent=True
                        )
                        emotion = emotion_result[0]['dominant_emotion']
                except Exception as e:
                    print(f"Emotion detection warning: {str(e)}")

                # Draw rectangle and label
                color = (0, 0, 255) if name == "Unknown" else (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label = f"{name} ({best_score:.2f}) - {emotion}"
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow('Face Recognition & Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

Loaded 10 images for Afnan
Loaded 9 images for rumana
Loaded 8 images for Shreya
Loaded 5 images for Smiksha

Loaded 4 persons.
